In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageOps, ImageFilter

class ImageApp:# a class with name ImageApp
    def __init__(self, root):
        self.root = root
        self.root.title("Image Processing App")
        
        self.root.geometry("800x600")# for Setting a fixed window size(800 x600 )well you can change your window size  as you wish)
        
        # setting a background color
        self.root.configure(bg='lightblue')
        
        # create a canvas for the background image (if needed)( a canva holds every component inside it )
        self.canvas = tk.Canvas(root, width=800, height=600, bg='lightblue', highlightthickness=0)
        self.canvas.pack(fill="both", expand=True)
        
        # create a frame for widgets(widgets are every individual components insside the canvas)
        self.frame = tk.Frame(self.canvas, bg='lightblue')
        self.frame.place(relwidth=1, relheight=1)
        
        # create a frame for the buttons
        self.button_frame = tk.Frame(self.frame, bg='lightblue')
        self.button_frame.pack(pady=10, fill='x')

        # load Image Button
        self.load_button = self.create_button(self.button_frame, text="Load Image", command=self.load_image, bg='blue', fg='white')
        self.load_button.pack(side='left', padx=5)

        # revert and Save Buttons
        self.revert_button = self.create_button(self.button_frame, text="Revert", command=self.revert_image, bg='green', fg='white')
        self.revert_button.pack(side='left', padx=5)

        self.save_button = self.create_button(self.button_frame, text="Save Image", command=self.save_image, bg='red', fg='white')
        self.save_button.pack(side='left', padx=5)

        # image Panel Canvas
        self.image_canvas = tk.Canvas(self.frame, bg='lightgray', width=700, height=400, scrollregion=(0, 0, 1000, 1000))
        self.image_canvas.pack(pady=10)

        # process Buttons Frame holds the buttons
        self.process_frame = tk.Frame(self.frame, bg='lightblue')
        self.process_frame.pack(pady=10, fill='x')

        # Process Buttons
        self.rotate_left_button = self.create_button(self.process_frame, text="Rotate Left", command=lambda: self.process_image('rotate_left'), bg='blue', fg='white')
        self.rotate_left_button.grid(row=0, column=0, padx=5, pady=5)

        self.rotate_right_button = self.create_button(self.process_frame, text="Rotate Right", command=lambda: self.process_image('rotate_right'), bg='blue', fg='white')
        self.rotate_right_button.grid(row=0, column=1, padx=5, pady=5)

        self.mirror_button = self.create_button(self.process_frame, text="Mirror", command=lambda: self.process_image('mirror'), bg='blue', fg='white')
        self.mirror_button.grid(row=0, column=2, padx=5, pady=5)

        self.bw_button = self.create_button(self.process_frame, text="Black & White", command=lambda: self.process_image('bw'), bg='blue', fg='white')
        self.bw_button.grid(row=0, column=3, padx=5, pady=5)

        self.sharpen_button = self.create_button(self.process_frame, text="Sharpen", command=lambda: self.process_image('sharpen'), bg='blue', fg='white')
        self.sharpen_button.grid(row=0, column=4, padx=5, pady=5)

        self.image = None
        self.original_image = None

    def create_button(self, parent, text, command, bg, fg):
        button = tk.Button(parent, text=text, command=command, bg=bg, fg=fg, relief='raised', font=('Arial', 12, 'bold'))
        button.bind("<Enter>", self.on_hover)
        button.bind("<Leave>", self.on_leave)
        button.bind("<ButtonPress>", self.on_click)
        button.bind("<ButtonRelease>", self.on_release)
        return button

    def on_hover(self, event):#on hover animations
        event.widget.config(bg='lightblue')

    def on_leave(self, event):
        event.widget.config(bg=event.widget.cget('bg'))

    def on_click(self, event):
        event.widget.config(bg='darkblue')

    def on_release(self, event):
        event.widget.config(bg='blue')

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.original_image = Image.open(file_path)
            self.image = self.original_image
            self.display_image(self.image)

    def display_image(self, img):
        # Resize image to fit within the canvas size while maintaining aspect ratio
        canvas_width = self.image_canvas.winfo_width()
        canvas_height = self.image_canvas.winfo_height()

        img_width, img_height = img.size
        aspect_ratio = img_width / img_height

        if img_width > canvas_width or img_height > canvas_height:
            if aspect_ratio > canvas_width / canvas_height:
                new_width = canvas_width
                new_height = int(canvas_width / aspect_ratio)
            else:
                new_height = canvas_height
                new_width = int(canvas_height * aspect_ratio)
        else:
            new_width, new_height = img_width, img_height

        resized_image = img.resize((new_width, new_height), Image.ANTIALIAS)
        tk_img = ImageTk.PhotoImage(resized_image)
        
        self.image_canvas.delete("all")
        self.image_canvas.create_image(0, 0, image=tk_img, anchor="nw")
        self.image_canvas.config(scrollregion=self.image_canvas.bbox("all"))  # Update scroll region if needed
        self.image_canvas.image = tk_img  # Keep a reference to avoid garbage collection

    def process_image(self, operation):
        if self.image:
            if operation == 'rotate_left':
                self.image = self.image.rotate(90, expand=True)
            elif operation == 'rotate_right':
                self.image = self.image.rotate(-90, expand=True)
            elif operation == 'mirror':
                self.image = ImageOps.mirror(self.image)
            elif operation == 'bw':
                self.image = ImageOps.grayscale(self.image)
            elif operation == 'sharpen':
                self.image = self.image.filter(ImageFilter.SHARPEN)
            self.display_image(self.image)
        else:
            messagebox.showwarning("Warning", "No image loaded")

    def revert_image(self):
        if self.original_image:
            self.image = self.original_image
            self.display_image(self.image)
        else:
            messagebox.showwarning("Warning", "No image to revert")

    def save_image(self):
        if self.image:
            file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                                   filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
            if file_path:
                self.image.save(file_path)
        else:
            messagebox.showwarning("Warning", "No image to save")

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageApp(root)
    root.mainloop()
